In [21]:
# Solution set for CS 155 Set 6, 2016/2017
# Authors: Fabian Boemer, Sid Murching, Suraj Nair

import numpy as np
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    first = reg * Ui
    second = Vj * (Yij - np.dot(Ui, Vj))
    grad = eta * (first - second)
    return grad

In [22]:
def load_data(filename):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, dtype = np.str , delimiter='\t')

In [23]:
def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    first = reg * Vj
    second = Ui * (Yij - np.dot(Ui, Vj))
    grad = eta* (first - second)
    return grad

In [24]:
def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    summation = 0
    U_val = np.linalg.norm(U)
    V_val = np.linalg.norm(V)
    for x in range (len(Y)): 
        i = Y[x][0] - 1
        j = Y[x][1] - 1
        yij = Y[x][2] - 1
        dot = np.dot(U[i], V[j])
        summation += (yij - dot) ** 2
    return (reg/2) * ((U_val)**2 + (V_val)**2) + (summation/2)

In [34]:
def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    U = np.random.uniform(-0.5, 0.5, (M, K))
    V = np.random.uniform(-0.5, 0.5, (N, K))
    first_loss = get_err(U, V, Y, reg)
    for epoch in range(max_epochs): 
        print("epoch: ", epoch)
        Y_shuffle = np.random.permutation(Y)
        prev_loss = get_err(U, V, Y, reg)
        for x in range (len(Y)): 
            i = Y_shuffle[x][0] - 1
            j = Y_shuffle[x][1] - 1
            y = Y_shuffle[x][2] - 1
            U[i] -= grad_U(U[i], y, V[j], reg, eta)
            V[j] -= grad_V(V[j], y, U[i], reg, eta)
        if (epoch == 0): 
            first_loss -= get_err(U, V, Y, reg)
        else: 
            error_diff = prev_loss - get_err(U, V, Y, reg)
            if (error_diff/first_loss) <= eps: 
                return (U, V, get_err(U, V, Y, reg)/len(Y))
    return (U, V, get_err(U, V, Y, reg)/len(Y)) 

In [51]:
Y_train = load_data("/Users/Netra/Documents/Caltech2ndYear/2ndTerm/CS155/HW/MiniProject2/data/train.txt")
Y_test = load_data("/Users/Netra/Documents/Caltech2ndYear/2ndTerm/CS155/HW/MiniProject2/data/test.txt")
Y_train = np.array([list(map(int, x)) for x in Y_train])
Y_test = np.array([list(map(int, x)) for x in Y_test])
M = max(max(Y_train[:,0]), max(Y_test[:,0])) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])) # movies
print("Factorizing with ", M, " users, ", N, " movies.")
Ks = [10,20,30,50,100]
reg = 0.0
eta = 0.03 # learning rate
E_in = []
E_out = []
Ein = 0
Eout = 0

Factorizing with  943  users,  1682  movies.


In [52]:
U,V, err = train_model(M, N, 20, eta, reg, Y_train)
Ein = err
Eout = get_err(U, V, Y_test)/len(Y_test)

epoch:  0
epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5
epoch:  6
epoch:  7
epoch:  8
epoch:  9
epoch:  10
epoch:  11
epoch:  12
epoch:  13
epoch:  14
epoch:  15
epoch:  16
epoch:  17


In [53]:
print("Ein: ", Ein)
print("Eout: ", Eout)

Ein:  0.22467468140622296
Eout:  0.644896601876968
